In [3]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

Links to the pages for scrapping :
    
    https://www.glassdoor.co.in/Job/bengaluru-data-scientist-jobs-SRCH_IL.0,9_IC2940587_KE10,24.htm
    https://www.glassdoor.co.in/Job/kolkata-data-scientist-jobs-SRCH_IL.0,7_IC2901633_KE8,22.htm
    https://www.glassdoor.co.in/Job/chennai-data-scientist-jobs-SRCH_IL.0,7_IC2833209_KE8,22.htm
    https://www.glassdoor.co.in/Job/pune-data-scientist-jobs-SRCH_IL.0,4_IC2856202_KE5,19.htm
    https://www.glassdoor.co.in/Job/mumbai-data-scientist-jobs-SRCH_IL.0,6_IC2851180_KE7,21.htm
    https://www.glassdoor.co.in/Job/hyderabad-data-scientist-jobs-SRCH_IL.0,9_IC2865319_KE10,24.htm
    https://www.glassdoor.co.in/Job/new-delhi-data-scientist-jobs-SRCH_IL.0,9_IC2891681_KE10,24.htm
    
    https://www.glassdoor.co.in/Job/india-data-scientist-jobs-SRCH_IL.0,5_IN115_KE6,20.htm
    


This code doesnt load several pages of the webbapplication, we need to modify this to do that so that we can scrape all the jobs in India

In [4]:
jobs = []
salary = {'Company Name':[],'Job Title':[],'Salary':[]}

In [5]:
def get_jobs(keyword, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="D:/2) Coding/Jupyter Notebook/DS_salary_proj/Scrapper and data/chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    
    keyword = '+'.join(keyword.split())
    url = 'https://www.glassdoor.co.in/Job/jobs.htm?sc.keyword='+ keyword      
    driver.get(url)
    time.sleep(5) #adding some time for the webpage to load
    
    #searching for the total number of jobs in the link
    num_jobs = int(''.join(driver.find_element_by_class_name("jobsCount").text.split()[0].split(','))) #modifiing to scrape all the jobs instead of a single job 
    print(num_jobs)
    
    #Going through each job in this page
    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.
 
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  
            
            print("Progress: {}".format("" + str(len(jobs)+1) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div> 
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1
                
            #used for debugging
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                
            jobs.append({"Job Title" : job_title,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs
            
            #-----------------------Added to extract salary from Indian company--------------------------#
            salary_company=[]
            jobtitle_company=[]
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="salary"]').click()
                
                try:
                    salaries = driver.find_elements_by_xpath('.//div[@class="hideHH alignRt avgSalaryCol"]//div[@class="strong"]')
                    salary_company = [a.text for a in salaries]  
                    jobtitles = driver.find_elements_by_xpath('.//div[@class="noPad expandHH"]//div[@class="jobTitle strong"]')
                    jobtitle_company = [a.text for a in jobtitles]
                except NoSuchElementException:
                    salary_company.append(-1)
                    jobtitle_company.append(-1)
                    
            except NoSuchElementException:  #Rarely, some job postings do not have the "salary" tab.
                salary_company.append(-1)
                jobtitle_company.append(-1)
            
            for i in range(len(jobtitle_company)):
                salary['Company Name'].append(company_name)
                salary['Job Title'].append(jobtitle_company[i])
                salary['Salary'].append(salary_company[i])
            
            
            # for debugging
            if verbose:
                print('list of the job titles: ',jobtitle_company)
                print('list of salaries: ',salary_company)
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
        
            #----------------------------------------------------------------------------------------------#                

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
            #we need to wait for JS to load
            time.sleep(2) 
        except NoSuchElementException:
            if num_jobs==len(jobs):
                print("Scraping over after reaching target number of jobs.\nNeeded : {}\ngot : {}.".format(num_jobs, len(jobs)))
            else:
                print("Scraping terminated before reaching target number of jobs.\nNeeded : {}\ngot : {}.".format(num_jobs, len(jobs)))
            break

#This line converts the dictionary object into a pandas DataFrame.

In [6]:
#This line will open a new chrome window and start the scraping.
get_jobs("Data Science", False)

2513
Progress: 1/2513
Progress: 2/2513
Progress: 3/2513
Progress: 4/2513
Progress: 5/2513
Progress: 6/2513
Progress: 7/2513
Progress: 8/2513
Progress: 9/2513
Progress: 10/2513
Progress: 11/2513
Progress: 12/2513
Progress: 13/2513
Progress: 14/2513
Progress: 15/2513
Progress: 16/2513
Progress: 17/2513
Progress: 18/2513
Progress: 19/2513
Progress: 20/2513
Progress: 21/2513
Progress: 22/2513
Progress: 23/2513
Progress: 24/2513
Progress: 25/2513
Progress: 26/2513
Progress: 27/2513
Progress: 28/2513
Progress: 29/2513
Progress: 30/2513
Progress: 31/2513


ElementClickInterceptedException: Message: element click intercepted: Element <li class="jl react-job-listing gdGrid selected" data-id="3466486336" data-emp-id="809791" data-is-organic-job="true" data-ad-order-id="4120" data-sgoc-id="-1" data-is-easy-apply="false" data-job-loc="Mumbai" data-job-loc-id="2851180" data-job-loc-type="C" data-njslv="false" style="">...</li> is not clickable at point (216, 278). Other element would receive the click: <div class="background-overlay" aria-label="Background Overlay"></div>
  (Session info: chrome=83.0.4103.61)


In [ ]:
df_jobs = pd.DataFrame(jobs) 
df_jobs

In [ ]:
df_salary = pd.DataFrame(salary) 
df_salary

In [ ]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="D:/2) Coding/Jupyter Notebook/scraping-glassdoor-selenium/chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    
    #old url
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    
    url = "https://www.glassdoor.co.in/Job/kolkata-data-scientist-jobs-SRCH_IL.0,7_IC2901633_KE8,22.htm"      
    driver.get(url)
    jobs = []
    time.sleep(5) #adding some time for the webpage to load
    
    #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_class_name("ModalStyle__xBtn___29PT9").click()  #clicking to the X.
        except NoSuchElementException:
            pass
        
        
    #Going through each job in this page
    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it
        #since I am waiting for the webpage to load I do not need to worry about the JS portion so timer here is lower and the loading is quicker 
        time.sleep(1)
        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)+1) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray small salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.